<a href="https://colab.research.google.com/github/slowvak/MIDeL/blob/main/DICOMnHL7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DICOM stands for Digital Imaging and Communications in Medicine. It is a standard format used for storing and transmitting medical images such as X-rays, CT scans, and MRIs. DICOM files contain both image data and metadata, which includes information about the patient, the imaging equipment used, and the imaging procedure itself 1.
DICOM images are used in medical image processing tasks such as segmentation, registration, and classification. Segmentation is the process of separating an image into multiple regions or objects. Registration is the process of aligning two or more images of the same subject taken at different times or with different imaging modalities. Classification is the process of assigning a label to an image based on its content 1.
Pydicom is a Python library that provides an easy-to-use interface for working with DICOM files. It allows you to read, modify, and write DICOM files using Python code. Pydicom provides access to important information elements such as patient name, patient ID, study date, study time, modality, and more 2. Here’s an example code snippet that demonstrates how to use pydicom to access these information elements:


In [ ]:
import pydicom

# Load a DICOM file
ds = pydicom.dcmread('path/to/dicom/file.dcm')

# Access important information elements
patient_name = ds.PatientName
patient_id = ds.PatientID
study_date = ds.StudyDate
study_time = ds.StudyTime
modality = ds.Modality

# Print the information elements
print(f'Patient Name: {patient_name}')
print(f'Patient ID: {patient_id}')
print(f'Study Date: {study_date}')
print(f'Study Time: {study_time}')
print(f'Modality: {modality}')


These are fairly straightforward and DICOM clearly specifies the format for representing date, time, and modality. The size and orientation of the pixels is a bit more complex--lets take a look.


In [ ]:
# Access important information elements
slice_thickness = ds.SliceThickness
pixel_spacing = ds.PixelSpacing
image_orientation = ds.ImageOrientationPatient

# Print the information elements
print(f'Slice Thickness: {slice_thickness}')
print(f'Pixel Spacing: {pixel_spacing}')
print(f'Image Orientation: {image_orientation}')


The slice thickness makes sense, and is always in millimeters, and is the distance from the ‘top’ of the slice to the ‘bottom’. Note this is NOT always the same as the slice spacing! Because DICOM is usually a 1-slice at a time data object, one must do much more work to figure out slice spacing. While it is common for all the slices in a series to have (nearly) the same spacing, that is not required, and was not as common in the early days of CT imaging. And since DICOM file names do not necessarily correlate with their position, you really should go through each file and sort them into slice position order and then figure out the spacing. That sounds simple, but again is not quite so easy since the slice position is (usually) a string with a letter indicating ‘L’, ‘R’,  ‘A’, ’P’, ’S’, or ‘I’, and then a number indicating how (in millimeters) far left or right or anterior, etc., it is from isocenter.
While slice thickness refers to the edges of the voxel, the pixel dimensions refer to the spacing between pixels in the X and Y dimensions, and this does expect the spacing to be constant for this image.
Now that we know about the size of the pixels, what about the orientation? Many imaging devices can acquire an image in almost any plane, but CT scanners nearly always (now) acquire axial images, and other planes are reconstructed from that. MRI is more variable, and for many organs, the images are acquired in an oblique orientation. Briefly about orientation names: ‘axial’ (also called ‘transverse’ or ‘transaxial’ means an image that is perpendicular to the long axis of the body. Furthermore, its left and right (‘X’) axis is also the left and right axis of the object being imaged, and the ‘Y’ axis of the image is considered to be in the anterior-posterior direction. A ‘Coronal’ image gets its name from the coronal suture, which is at the top of the head and is a plane that is like facing the patient: the ‘X’ axis of the image is also the ‘X’ axis of the patient, but the ‘Y’ axis of the image is the ‘Z’ axis (inferior-superior) of the patient. Sagittal gets its name from the sagittal suture, which is the connection between the left and right halves of the skull. Therefore, its ‘X’ axis is the anterior-posterior line of the patient, and its ‘Y’ axis is the  ‘Z’ axis of the patient. The relationship between moving 1 pixel along the ‘X’ axis of the image is thus represented as 3 floating point values: the change in ‘X’ of the patient, the change in ‘Y’ of the patient, and the change in ‘Z’ of the patient. Similarly, there is another set of 3 values for showing the change as one moves along the ‘Y’ axis of the image, again representing the ‘X’, ‘Y’, and ‘Z’ change in the patient. This set of 6 values completely defines the orientation of the image, and they are known as directional cosines. From these, one can compute the orientation of the slice:


In [ ]:

def getMajorAxisFromDirCos(x, y, z):
    axis = ""
    if (x < 0):
        XOrient = "R"
    else:
        XOrient = "L"
    if (y < 0):
        YOrient = "A"
    else:
        YOrient = "P"
    if (z < 0):
        ZOrient = "F"
    else:
        ZOrient = "H"

    absX = math.fabs(x)
    absY = math.fabs(y)
    absZ = math.fabs(z)

    if ((absX > 0.25) and (absX > absY) and (absX > absZ)):
        axis = XOrient
    elif ((absY > 0.25) and (absY > absX) and (absY > absZ)):
        axis = YOrient
    elif ((absZ > 0.25) and (absZ > absX) and (absZ > absY)):
        axis = ZOrient
    return axis


def getImageOrientFromDirCos(rowX, rowY, rowZ, colX, colY, colZ):
    label = ""
    rowAxis = getMajorAxisFromDirCos(rowX, rowY, rowZ)
    colAxis = getMajorAxisFromDirCos(colX, colY, colZ)

    if (rowAxis != "" and colAxis != ""):
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "A" or colAxis == "P")):
            label = "AXL"
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "A" or colAxis == "P")):
            label = "AXL"

        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "H" or colAxis == "F")):
            label = "COR"
        if ((rowAxis == "R" or rowAxis == "L") and (colAxis == "H" or colAxis == "F")):
            label = "COR"

        if ((rowAxis == "A" or rowAxis == "P") and (colAxis == "H" or colAxis == "F")):
            label = "SAG"
        if ((rowAxis == "A" or rowAxis == "P") and (colAxis == "H" or colAxis == "F")):
            label = "SAG"
    else:
        label = "OBL"
    return label

ImageOrientDirCos = ds[0x20, 0x0037].value
Orient = getImageOrientFromDirCos(ImageOrientDirCos[0], ImageOrientDirCos[1], ImageOrientDirCos[2],
                                  ImageOrientDirCos[3], ImageOrientDirCos[4], ImageOrientDirCos[5])



The main difference between DICOM and other image formats is that DICOM groups information into data sets. A DICOM file consists of several tags, all packed into a single file. It stores information such as demographic details about the patient, imaging study’s acquisition parameters, image dimensions, matrix size, color space, and more. This makes it easier to manage and exchange medical images between different devices from multiple manufacturers.

A great site for looking up DICOM tags is: http://dicomlookup.com/